In this project, we will train a GNN to perform link prediction on a heterogenous graph from the Spotify Million Playlists dataset.

# Import libraries

In [ ]:
# !pip install numpy iprogress tqdm networkx torch_geometric
# Server IP: http://192.168.0.103:8888/?token=klobasa

In [6]:
# temporary imports (delete later)
from pprint import pprint
import random
import numpy as np
import time

In [ ]:
import config
import loader
from model import *

# Load datasets

In [9]:
# !rm ../spotify_million_playlist_dataset/pickles/datasets_example.pkl
# !rm ../spotify_million_playlist_dataset/pickles/G_example.pkl
# !rm ../spotify_million_playlist_dataset/pickles/ghetero_example.pkl
# !ls ../spotify_million_playlist_dataset/pickles/
# del G
# del ghetero
# del datasets

In [11]:
ghetero = loader.get_ghetero()
datasets = loader.get_datasets()
data_train, data_val, data_test = datasets
print("Finished loading data.")

Loading ghetero from pickle ...
Loading datasets from pickle ...
Finished loading data.


# Processing

In [13]:
# create training mask for playlist nodes
train_mask = torch.zeros(ghetero["playlist"].x.shape[0], dtype=torch.bool)
train_mask[torch.randperm(train_mask.shape[0])[:int(train_mask.shape[0]*0.8)]] = True

ghetero["playlist"].train_mask = train_mask

ghetero["playlist"].y = torch.LongTensor([1]*ghetero["playlist"].x.shape[0])

In [14]:
model = HeteroModel(64, ghetero.x_dict, ghetero.metadata())
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [15]:
edge_label_index = data_train["track", "contains", "playlist"].edge_label_index
edge_label = data_train["track", "contains", "playlist"].edge_label
train_loader = torch_geometric.loader.LinkNeighborLoader(
    data=data_train,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("track", "contains", "playlist"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

In [17]:
train(model, train_loader, optimizer)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)